# GPU 사용가능한지 확인

In [1]:
import torch

# GPU 사용 가능 -> True, GPU 사용 불가 -> False
print(torch.cuda.is_available())

True


# YOlov8n모델 예측 후 객체의 정보 얻기

In [2]:
from ultralytics import YOLO
import torch
# Load a model
model = YOLO('yolov8n.pt')  # pretrained YOLOv8n model

# Run batched inference on a list of images
path = './12_13_67_thumbnail.jpg'
results = model.predict(path, save=True)  # return a generator of Results objects

# Process results generator
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs

# 클래스 id
object_type = []
for _ in boxes.cls:
    object_type.append(int(_))

# 객체의 좌표
yolo_point = []
for _ in boxes.xywh:
    center_x = int(_[0])
    center_y = int(_[1])
    width = int(_[2])
    hight = int(_[3])

    yolo_point.append((center_x, center_y, width, hight))


image 1/1 d:\yolo\12_13_67_thumbnail.jpg: 640x640 4 persons, 1 bicycle, 2 cars, 2 motorcycles, 15.0ms
Speed: 3.0ms preprocess, 15.0ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict19


In [3]:
result_path = './' + result.save_dir.replace('\\', '/')
result_path

'./runs/detect/predict19'

In [4]:
# coco 데이터 구조 파악
result.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [5]:
object_type

[2, 2, 0, 0, 3, 0, 0, 1, 3]

In [6]:
len(object_type), len(yolo_point)

(9, 9)

In [7]:
object_type

[2, 2, 0, 0, 3, 0, 0, 1, 3]

In [8]:
yolo_point

[(560, 355, 159, 174),
 (381, 291, 42, 24),
 (314, 296, 24, 73),
 (430, 313, 35, 124),
 (407, 319, 30, 42),
 (462, 322, 35, 117),
 (270, 296, 25, 59),
 (237, 327, 25, 53),
 (404, 320, 24, 34)]

### 이미지내 불필요한 클래스 제거

In [9]:
# 불필요한 클래스id 인덱스 받기
drop_index = []

# 0 : 사람, 1 : 자전거, 2 : 자동차
# 3 : 오토바이, 5 : 버스, 7 : 트럭
for _ in range(0, len(object_type)):
    if object_type[_] == 0 or object_type[_] == 1 or object_type[_] == 2 or object_type[_] == 3 or object_type[_] == 5 or object_type[_] == 7:
        pass
    else:
        drop_index.append(_)

# 불필요한 클래스 인덱스 삭제
# 인덱스 번호를 안 뒤집을 경우 앞에서부터 삭제하므로
# 뒤로갈수록 해당 인덱스 존재 안해서.
drop_index.reverse()
for _ in drop_index:
    object_type.pop(_)
    yolo_point.pop(_)

### 클래스 id 정수형 -> 한글로 변환

In [10]:
# 클래스id -> 한글로 변환
for _ in range(0, len(object_type)):
    if object_type[_] == 0:
        object_type[_] = '사람'

    elif object_type[_] == 1:
        object_type[_] = '자전거'

    elif object_type[_] == 2:
        object_type[_] = '자동차'

    elif object_type[_] == 3:
        object_type[_] = '오토바이'

    elif object_type[_] == 5:
        object_type[_] = '버스'

    elif object_type[_] == 7:
        object_type[_] = '트럭'       

object_type


['자동차', '자동차', '사람', '사람', '오토바이', '사람', '사람', '자전거', '오토바이']

### Yolo식 좌표는 center_x, center_y, w, h -> 바운딩박스 4개의 점 좌표로 변환

In [11]:
# 사각형의 4개의 점 좌표 구하기.
# x1 : 왼쪽 상단, x2 : 오른쪽 상단
# x3 : 왼쪽 하단, x4 : 오른쪽 하단

# 하나의 클래스 기준 4개의 점 좌표 저장.
point = []

for raw in yolo_point:
    x = raw[0]
    y = raw[1]
    w = raw[2]
    h = raw[3]
    x1 = (int(x - (w/2)), int(y + (h/2)))
    x2 = (int(x + (w/2)), int(y + (h/2)))
    x3 = (int(x - (w/2)), int(y - (h/2)))
    x4 = (int(x + (w/2)), int(y - (h/2)))

    point.append((x1, x2, x3, x4))

In [12]:
point

[((480, 442), (639, 442), (480, 268), (639, 268)),
 ((360, 303), (402, 303), (360, 279), (402, 279)),
 ((302, 332), (326, 332), (302, 259), (326, 259)),
 ((412, 375), (447, 375), (412, 251), (447, 251)),
 ((392, 340), (422, 340), (392, 298), (422, 298)),
 ((444, 380), (479, 380), (444, 263), (479, 263)),
 ((257, 325), (282, 325), (257, 266), (282, 266)),
 ((224, 353), (249, 353), (224, 300), (249, 300)),
 ((392, 337), (416, 337), (392, 303), (416, 303))]

In [13]:
point[0]

((480, 442), (639, 442), (480, 268), (639, 268))

### 알고리즘 구현

In [14]:
# opencv식 워닝존 좌표
warning_point = [(270, 310), (12, 445), (610, 445), (383, 310)]

# 변환 워닝존 좌표
warning_point = [(12, 310), (270, 445), (383, 445), (610, 310)]

In [15]:
point[0]

((480, 442), (639, 442), (480, 268), (639, 268))

In [16]:
x = point[2][0][0]
y = point[2][0][1]

In [17]:
max_x = warning_point[3][0]
min_x = warning_point[0][0]

max_y = warning_point[2][1]
min_y = warning_point[0][1]

In [18]:
x = 13
y = 350

In [19]:
min_x < x <max_x and min_y < y < max_y

True

In [20]:
min_x, max_x

(12, 610)

In [21]:
# !pip install shapely

In [22]:
from shapely.geometry import Polygon, Point

def warning_zone(point, polygon):
    """
    point: [x, y] 좌표 값을 가진 리스트
    polygon: 다각형의 꼭지점 좌표를 가진 리스트
    """

    # 점의 좌표를 사용하여 점의 객체를 만듭니다.
    point_obj = Point(point)

    # 다각형의 꼭지점 좌표를 사용하여 다각형의 경계를 구합니다.
    polygon_path = Polygon(polygon)

    # 점의 객체가 다각형의 경계 안에 있는지 확인합니다.
    return polygon_path.contains(point_obj)

# 워닝존
rectangle = [(12, 310), (270, 445), (383, 445), (610, 310)]

# 데이터 클래스 하나 4개의 점
# square = point

result_case = []
# 좌표가 사다리꼴 안에 있는지 확인합니다.
for square, result_type in zip(point, object_type):
    for p in square:
        point_obj = Point(p)
        if warning_zone(point_obj, rectangle):
            # print(_, "포함됨")
            # 조건문
            if p[0] < int((rectangle[3][0] - rectangle[0][0]) / 2):
                result_txt = '왼쪽'
                print(f'{result_txt}에 {result_type}있습니다.')
                result_case.append([result_txt, result_type])
                break
            elif int((rectangle[3][0] - rectangle[0][0]) / 2) < p[0]:
                result_txt = '오른쪽'
                result_case.append([result_txt, result_type])
                print(f'{result_txt}에 {result_type}있습니다.')
                break
        else:
            pass
            # print("포함되지 않음")

result_case


오른쪽에 사람있습니다.
오른쪽에 사람있습니다.
오른쪽에 오토바이있습니다.
오른쪽에 사람있습니다.
왼쪽에 사람있습니다.
왼쪽에 자전거있습니다.
오른쪽에 오토바이있습니다.


[['오른쪽', '사람'],
 ['오른쪽', '사람'],
 ['오른쪽', '오토바이'],
 ['오른쪽', '사람'],
 ['왼쪽', '사람'],
 ['왼쪽', '자전거'],
 ['오른쪽', '오토바이']]

In [ ]:
for _ in range(0, len(result_case)):
    a = result_case[_][0]
    ob = result_case[_][1]

이미지 테스트

In [23]:
import os
import cv2
import numpy as np
import sys

filePath  = './runs/detect/predict14/12_13_67_thumbnail.jpg'

img = cv2.imread(filePath)

w, h, c = img.shape

mask = np.zeros((w,h,c), dtype=np.uint8)

pts1 = np.array([[270, 310], [12, 445], [610, 445], [383, 310]])
# pts1 = np.array([[12, 310], [270, 445], [383, 445], [610, 310]])

desired_alpha = 90 
alpha = desired_alpha / 255.0 

# 다각형 그리기
polyline = cv2.polylines(mask, [pts1], isClosed=True, color=(0, 0, 255), thickness=5)

# 마스크 채우기
mask1 = cv2.fillPoly(polyline, [pts1], (255,0,0))
img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
mask1 = cv2.cvtColor(mask1, cv2.COLOR_BGR2BGRA)
result = cv2.addWeighted(img, 1, mask1, alpha, 0)

cv2.imshow('img', result)
cv2.waitKey(0)
cv2.destroyAllWindows()